In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO(r"E:\best.pt")
results = model.predict(source="0", show=True)
print(results)

In [ ]:
import cv2
import torch
import requests
from ultralytics import YOLO

# Function to send command to ESP32
def send_command(command):
    data = command + '\n'
    headers = {'Content-Type': 'text/plain'}
    response = requests.post(url, data=data)
    print("Response from ESP32:", response.text)
    if response.text == "success":
        return 1
    else:
        return 0

# ESP IP URL
url = 'http://192.168.104.99/data'

# URL for the stream
stream_url = "http://100.77.46.130:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Collect results and send labels to ESP32
results_count = 0
while results_count < 10:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640, 480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)

    max_confidence_label = 'P'  # Default label is 'P' (plastic) for demonstration purposes
    max_confidence = 0.0

    for result in results:
        for prediction in result:
            # Check if the prediction is not an empty list and contains necessary elements
            if len(prediction) >= 6:  # Check the minimum length required
                label_id = prediction[5]  # Accessing the class ID
                confidence = prediction[4]  # Accessing the confidence value

                # Logic to assign labels based on label IDs
                if confidence > max_confidence:
                    if label_id == 1:
                        max_confidence_label = 'O'
                    elif label_id == 0:
                        max_confidence_label = 'B'
                    elif label_id == 2:
                        max_confidence_label = 'M'
                    elif label_id == 3:
                        max_confidence_label = 'P'
                    max_confidence = confidence

    # Print the label with maximum confidence before sending to ensure correctness
    print("Label sent to ESP32:", max_confidence_label)

    # Send label with maximum confidence to ESP32
    send_command(max_confidence_label)

    # Increment the results count for each detection
    results_count += 1

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# URL for the stream
stream_url = "http://192.168.137.94:8080/video"

# Video capture object
cap = cv2.VideoCapture(stream_url)

# Check if the stream is opened correctly
if not cap.isOpened():
    print("Error opening video stream or file")

# YOLO model initialization
model = YOLO(r"E:\best.pt")

# Read and display the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (640,480))

    # Perform object detection on the resized frame
    results = model.predict(resized_frame, show=True)
    print(results)

    # Display the original frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
model = YOLO(r"E:\best.pt")
model.export(format="saved_model")

In [ ]:
import cv2

# Initialize the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera or specify the camera index

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Display the frame
    cv2.imshow("Camera Stream", frame)

    # Check for a key press and break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Exit the loop when 'q' is pressed

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
